In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, hinge_loss
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [2]:
data = pd.read_csv("/letter-recognition.data", delimiter = ',') 
data.columns =['Label',	'1',	'2',	'3',	'4',	'5',	'6',	'7',	'8',	'9',	'10',	'11',	'12',	'13',	'14',	'15',	'16']
print(data)

      Label  1   2  3  4  5   6   7  8  9  10  11  12  13  14  15  16
0         I  5  12  3  7  2  10   5  5  4  13   3   9   2   8   4  10
1         D  4  11  6  8  6  10   6  2  6  10   3   7   3   7   3   9
2         N  7  11  6  6  3   5   9  4  6   4   4  10   6  10   2   8
3         G  2   1  3  1  1   8   6  6  6   6   5   9   1   7   5  10
4         S  4  11  5  8  3   8   8  6  9   5   6   6   0   8   9   7
...     ... ..  .. .. .. ..  ..  .. .. ..  ..  ..  ..  ..  ..  ..  ..
19994     D  2   2  3  3  2   7   7  7  6   6   6   4   2   8   3   7
19995     C  7  10  8  8  4   4   8  6  9  12   9  13   2   9   3   7
19996     T  6   9  6  7  5   6  11  3  7  11   9   5   2  12   2   4
19997     S  2   3  4  2  1   8   7  2  6  10   6   8   1   9   5   8
19998     A  4   9  6  6  2   9   5  3  1   8   1   8   2   7   2   8

[19999 rows x 17 columns]


In [4]:
indexing = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'I':8, 'J':9, 'K':10, 'L':11, 'M':12, 'N':13, 'O':14, 'P':15, 'Q':16, 'R':17, 'S':18, 'T':19, 'U':20, 'V':21, 'W':22, 'X':23, 'Y':24, 'Z':25}

In [5]:
def Markov_sampling_SVM(Train,n,k,q,n2,Kernel):

  # step 1: draw randomly N sample
  Random_N_sample=Train.sample(n)

  #Train SVMC using differnt Kernel
  X_Random_N_sample=Random_N_sample.drop('Label',axis=1)
  Y_Random_N_sample=Random_N_sample['Label']

  if Kernel=="linear":
    Model_Sample=SVC(kernel = Kernel)
  if Kernel=="rbf":
    Model_Sample=SVC(kernel = Kernel,random_state=0)
  if Kernel=="sigmoid":
    Model_Sample=SVC(kernel = Kernel)
  if Kernel=="poly":
    Model_Sample=SVC(kernel = Kernel,degree = 2)

  # f0
  Model_Sample.fit(X_Random_N_sample,Y_Random_N_sample)

  #step 3
  N=[]
  i=0

  N.append(0)
  t=1  

  #draw randomly sample zt

  zt=Train.sample()
  if (N[i]%2==0):
    N[i]=N[i]+1

  Markov_sample=[]   # for storing the accepted zstar

    #step 4
  cnt=0
  while True:

    zstar=Train.sample()

    #calculation of alpha or acceptance probablity
    zt_x=zt.drop('Label', axis=1)
    zt_y=zt['Label']
    zt_decison = Model_Sample.decision_function(zt_x) 
    zt_pre = max(zt_decison[0]) 

    zstar_x=zstar.drop('Label', axis=1)
    zstar_y=zstar['Label']
    zstar_decison = Model_Sample.decision_function(zstar_x)  
    zstar_pre = max(zstar_decison[0]) 

    alpha = np.exp(np.log2(abs(zstar_pre/zt_pre)))

    #step 5

    if round(alpha,2)==1:
      Markov_sample.append(zstar)
      cnt=0
    elif alpha > 1:
      Markov_sample.append(zstar)
      cnt=0
    elif cnt==n2:
      Markov_sample.append(zstar)
      cnt=0
    else:
      cnt=cnt+1
      continue
      
    N[i]=N[i]+1
    zt=zstar

    if N[i]<n/2:
      continue
    else :
      break

    #Step 6

  Markov_sample = pd.concat(Markov_sample)
  #Train N Markov  
  X_Markov_sample=Markov_sample.drop('Label',axis=1)
  Y_Markov_sample=Markov_sample['Label']

  if Kernel=="linear":
    Markov_Model_Sample=SVC(kernel = Kernel)
  if Kernel=="rbf":
    Markov_Model_Sample=SVC(kernel = Kernel,random_state=0)
  if Kernel=="sigmoid":
    Markov_Model_Sample=SVC(kernel = Kernel)
  if Kernel=="poly":
    Markov_Model_Sample=SVC(kernel = Kernel,degree = 2)


  Markov_Model_Sample.fit(X_Markov_sample,Y_Markov_sample)

  #step 7
  return Markov_Model_Sample

In [6]:
len = round(data.shape[0]*0.8)
train, test = data[:len], data[len:]

In [7]:
test_x = test.drop('Label', axis=1)
test_y = test['Label']

In [9]:
Markov_model = Markov_sampling_SVM(train, 100, 2, 1.2, 10, "linear")
Markov_model_prediction = Markov_model.predict(test_x)
print(classification_report(test_y, Markov_model_prediction),"\n")
linear_kernel= metrics.accuracy_score(test_y, Markov_model_prediction)
print("accuracy", metrics.accuracy_score(test_y, Markov_model_prediction), "\n")

              precision    recall  f1-score   support

           A       0.58      0.44      0.50       156
           B       0.35      0.15      0.21       136
           C       0.28      0.25      0.26       142
           D       0.00      0.00      0.00       167
           E       0.20      0.09      0.12       152
           F       0.10      0.13      0.12       153
           G       0.07      0.04      0.05       164
           H       0.08      0.12      0.10       151
           I       0.63      0.56      0.59       165
           J       0.87      0.31      0.46       148
           K       0.09      0.21      0.13       146
           L       0.82      0.63      0.71       157
           M       0.78      0.63      0.70       144
           N       0.54      0.25      0.34       166
           O       0.15      0.64      0.25       139
           P       0.54      0.58      0.56       168
           Q       0.38      0.10      0.15       168
           R       0.18    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
Markov_model = Markov_sampling_SVM(train, 100, 2, 1.2, 10, "rbf")
Markov_model_prediction = Markov_model.predict(test_x)
print(classification_report(test_y, Markov_model_prediction),"\n")
rbf_kernel= metrics.accuracy_score(test_y, Markov_model_prediction)
print("accuracy", metrics.accuracy_score(test_y, Markov_model_prediction), "\n")

              precision    recall  f1-score   support

           A       0.92      0.54      0.69       156
           B       0.00      0.00      0.00       136
           C       0.74      0.14      0.24       142
           D       0.00      0.00      0.00       167
           E       0.00      0.00      0.00       152
           F       0.15      0.73      0.26       153
           G       0.00      0.00      0.00       164
           H       0.08      0.30      0.13       151
           I       0.00      0.00      0.00       165
           J       0.08      0.12      0.10       148
           K       0.06      0.59      0.12       146
           L       0.89      0.61      0.72       157
           M       1.00      0.01      0.01       144
           N       0.00      0.00      0.00       166
           O       0.00      0.00      0.00       139
           P       0.00      0.00      0.00       168
           Q       0.00      0.00      0.00       168
           R       0.70    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
Markov_model = Markov_sampling_SVM(train, 100, 2, 1.2, 10, "sigmoid")
Markov_model_prediction = Markov_model.predict(test_x)
print(classification_report(test_y, Markov_model_prediction),"\n")
sigmoid_kernel= metrics.accuracy_score(test_y, Markov_model_prediction)
print("accuracy", metrics.accuracy_score(test_y, Markov_model_prediction), "\n")

              precision    recall  f1-score   support

           A       0.00      0.00      0.00       156
           B       0.04      0.24      0.06       136
           C       0.00      0.00      0.00       142
           D       0.00      0.00      0.00       167
           E       0.06      0.38      0.10       152
           F       0.00      0.00      0.00       153
           G       0.00      0.00      0.00       164
           H       0.00      0.00      0.00       151
           I       0.00      0.00      0.00       165
           J       0.00      0.00      0.00       148
           K       0.09      0.18      0.12       146
           L       0.00      0.00      0.00       157
           M       0.06      0.61      0.10       144
           N       0.00      0.00      0.00       166
           O       0.00      0.00      0.00       139
           P       0.00      0.00      0.00       168
           Q       0.00      0.00      0.00       168
           R       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
Markov_model = Markov_sampling_SVM(train, 100, 2, 1.2, 10, "poly")
Markov_model_prediction = Markov_model.predict(test_x)
print(classification_report(test_y, Markov_model_prediction),"\n")
poly_kernel= metrics.accuracy_score(test_y, Markov_model_prediction)
print("accuracy", metrics.accuracy_score(test_y, Markov_model_prediction), "\n")

              precision    recall  f1-score   support

           A       0.73      0.72      0.72       156
           B       0.00      0.00      0.00       136
           C       0.88      0.15      0.25       142
           D       0.22      0.59      0.32       167
           E       0.20      0.32      0.24       152
           F       0.28      0.10      0.15       153
           G       0.00      0.00      0.00       164
           H       0.00      0.00      0.00       151
           I       0.38      0.09      0.15       165
           J       1.00      0.28      0.43       148
           K       0.15      0.20      0.17       146
           L       0.93      0.55      0.69       157
           M       1.00      0.35      0.52       144
           N       0.22      0.34      0.27       166
           O       0.12      0.36      0.18       139
           P       0.43      0.54      0.48       168
           Q       0.10      0.08      0.09       168
           R       0.13    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
print("Misclassification rate\n")
print("linear_kernel     : ",linear_kernel)
print("rbf_kernel        : ",rbf_kernel)
print("sigmoid_kernel    : ",sigmoid_kernel)
print("polynomial_kernel : ",poly_kernel)

Misclassification rate

linear_kernel     :  0.1825
rbf_kernel        :  0.1585
sigmoid_kernel    :  0.066
polynomial_kernel :  0.3375
